In [1]:
__NAME = 'Harmonic 2'
# root_path = '/content/drive/MyDrive/Colab Notebooks/Code/Harmonic Oscillator'
# %tensorflow_version 1.x
# !pip install pyDOE

In [2]:
# from google.colab import drive
# drive.mount('/content/drive')

In [3]:
# import os
# os.chdir(root_path)
# !pwd

# Imports

In [4]:
import tensorflow as tf
print(tf.version)
print(tf.test.is_built_with_cuda())
print(tf.test.is_gpu_available(cuda_only=False, min_cuda_compute_capability=None))

<module 'tensorflow._api.v1.version' from 'I:\\Users\\Ragav\\miniconda3\\envs\\juptenflowgpu115\\lib\\site-packages\\tensorflow_core\\_api\\v1\\version\\__init__.py'>
True
True


In [5]:
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D
# from plotting import newfig, savefig
import matplotlib.gridspec as gridspec
from mpl_toolkits.axes_grid1 import make_axes_locatable

In [6]:
import numpy as np
import scipy.io
from scipy.interpolate import griddata
import time
from pyDOE import lhs

In [7]:
import pickle as pkl

In [8]:
%matplotlib widget

# Eqn

In [9]:
k = 1

$$
\Psi'' - k^2 \Psi = 0
$$

# Model 

In [10]:
# Layers
u_layers = [1, 50, 50, 50, 50, 1]
pde_layers = [3, 100, 100, 1]

layers = [1, 50, 50, 50, 50, 1]

# layers = [1, 100, 1000, 1000, 100, 1]

# layers = [1, 3, 3, 3, 1]

In [11]:
x_tf = tf.placeholder(tf.float32, shape=[None, 1])
u_tf = tf.placeholder(tf.float32, shape=[None, 1])
# v_tf = tf.placeholder(tf.float32, shape=[None, 1])
x_tf, u_tf

(<tf.Tensor 'Placeholder:0' shape=(?, 1) dtype=float32>,
 <tf.Tensor 'Placeholder_1:0' shape=(?, 1) dtype=float32>)

In [12]:
def initialize_NN(layers):
    weights = []
    biases = []
    num_layers = len(layers)
    for l in range(0, num_layers - 1):
        W = xavier_init(size=[layers[l], layers[l + 1]])
        b = tf.Variable(tf.zeros([1, layers[l + 1]], dtype=tf.float32),
                        dtype=tf.float32)
        weights.append(W)
        biases.append(b)
    return weights, biases

def xavier_init(size):
    in_dim = size[0]
    out_dim = size[1]
    xavier_stddev = np.sqrt(2 / (in_dim + out_dim))
    return tf.Variable(tf.truncated_normal([in_dim, out_dim],
                                           stddev=xavier_stddev,
                                           dtype=tf.float32),
                       dtype=tf.float32)

In [13]:
def neural_net(X, weights, biases):
    num_layers = len(weights) + 1
    H = X
    for l in range(0, num_layers - 2):
        W = weights[l]
        b = biases[l]
        H = tf.sin(tf.add(tf.matmul(H, W), b))
    W = weights[-1]
    b = biases[-1]
    Y = tf.add(tf.matmul(H, W), b)
    return Y

In [14]:
u_weights, u_biases = initialize_NN(layers)
# v_weights, v_biases = initialize_NN(layers)

In [15]:
# Load Weights and biases

# npz1 = np.load('Harmonic 1\\weights.pkl')
# npz2 = np.load('Harmonic 1\\biases.pkl')

with open(__NAME + '\\weights.pkl', 'rb') as db_file:
    W_pkl = pkl.load(db_file)

with open(__NAME + '\\biases.pkl', 'rb') as db_file:
    B_pkl = pkl.load(db_file)

W = []
B = []
for w, b in zip(W_pkl, B_pkl):
    W.append(tf.Variable(w))
    B.append(tf.Variable(b))

u_weights = W
u_biases = B

In [16]:
lb_tf = tf.placeholder(tf.float32, shape=[1])
ub_tf = tf.placeholder(tf.float32, shape=[1])
# itb_tf = tf.placeholder(tf.float32, shape=[1])

In [17]:
# tf placeholders for Solution
x_f_tf = tf.placeholder(tf.float32, shape=[None, 1])

x0_tf = tf.placeholder(tf.float32, shape=[None, 1])
u0_tf = tf.placeholder(tf.float32, shape=[None, 1])
# v0_tf = tf.placeholder(tf.float32, shape=[None, 1])

u_x0_tf = tf.placeholder(tf.float32, shape=[None, 1])
# v_x0_tf = tf.placeholder(tf.float32, shape=[None, 1])

In [18]:
def sol_net_u(x):
    X = tf.concat([x], 1)
    H = 2.0 * (X - lb_tf) / (ub_tf - lb_tf) - 1.0
    u = neural_net(H, u_weights, u_biases)
    # v = neural_net(H, v_weights, v_biases)
    u_x = tf.gradients(u, x)[0]
    # v_x = tf.gradients(v, x)[0]
    return u, u_x

def sol_net_f_R1(x):
    u, u_x = sol_net_u(x)

    u_xx = tf.gradients(u_x, x)[0]
    # v_xx = tf.gradients(v_x, x)[0]

    f = u_xx - k**2 * u
    # g = v_xx + k2_1 * v

    return f

def sol_net_f_R2(x):
    u, u_x = sol_net_u(x)

    u_xx = tf.gradients(u_x, x)[0]
    # v_xx = tf.gradients(v_x, x)[0]

    f = u_xx - u
    # g = v_xx + k2_2 * v

    return f

In [19]:
# tf graphs for Solution
u0_pred, u_x0_pred = sol_net_u(x0_tf)

sol_f_pred = sol_net_f_R1(x_f_tf)

In [20]:
# loss for Solution
sol_loss = tf.reduce_sum(tf.square(u0_tf - u0_pred)) + \
           tf.reduce_sum(tf.square(u_x0_pred - u_x0_tf)) + \
           tf.reduce_sum(tf.square(sol_f_pred))
        #    tf.reduce_sum(tf.square(v0_tf - v0_pred)) + \
        #    tf.reduce_sum(tf.square(v_x0_pred - v_x0_tf)) + \
        #    tf.reduce_sum(tf.square(sol_g_pred))

In [21]:
adam_optimizer = tf.train.AdamOptimizer()
sol_train_op_Adam = adam_optimizer.minimize(
            sol_loss,
            var_list= u_weights + u_biases)

In [22]:
# Optimizer for Solution
sol_optimizer = tf.contrib.opt.ScipyOptimizerInterface(
    sol_loss,
    var_list=u_weights + u_biases,
    method='L-BFGS-B',
    options={
        'maxiter': 50000,
        'maxfun': 50000,
        'maxcor': 50,
        'maxls': 50,
        'ftol': 1.0 * np.finfo(float).eps
    })

The TensorFlow contrib module will not be included in TensorFlow 2.0.
For more information, please see:
  * https://github.com/tensorflow/community/blob/master/rfcs/20180907-contrib-sunset.md
  * https://github.com/tensorflow/addons
  * https://github.com/tensorflow/io (for I/O related ops)
If you depend on functionality not listed there, please file an issue.



In [23]:
# tf session
sess = tf.Session(config=tf.ConfigProto(
    allow_soft_placement=True, log_device_placement=True))
init = tf.global_variables_initializer()
sess.run(init)

Device mapping:
/job:localhost/replica:0/task:0/device:GPU:0 -> device: 0, name: NVIDIA GeForce GTX 1650 Ti, pci bus id: 0000:01:00.0, compute capability: 7.5



# Training
## Prepare data

In [24]:
lb = np.array([0])
ub = np.array([np.pi * 2])

In [25]:
x0 = np.array([0])

u0 = np.array([3])
# v0 = np.array([0])

u_x0 = np.array([- 3])
# v_x0 = np.array([k2_1 * (B - A)])

In [26]:
N_f = 1000
(N_f)

1000

In [27]:
X_f_train = lb + (ub - lb) * lhs(1, N_f)

# X_f_train

In [28]:
tf_dict = {
    lb_tf: lb,
    ub_tf: ub,
    x0_tf: x0[None, :],
    u0_tf: u0[None, :],
    # v0_tf: v0[None, :],
    u_x0_tf: u_x0[None, :],
    # v_x0_tf: v_x0[None, :],
    x_f_tf: X_f_train,
}

In [29]:
start_time = time.time()
it = 0
end = True
while not(end):

    sess.run(sol_train_op_Adam, tf_dict)

    # Print
    if it % 10 == 0:
        elapsed = time.time() - start_time
        loss_value = sess.run(sol_loss, tf_dict)
        print('It: %d, Loss: %.3e, Time: %.2f' %
                (it, loss_value, elapsed))
        start_time = time.time()

        if (loss_value < 5 * 10**(-2)) or (it > 20000):
            end = True

    it = it + 1

In [30]:
def callback(loss):
    print('Loss: %e' % (loss))

In [31]:
# sol_optimizer.minimize(sess,
#                        feed_dict=tf_dict,
#                        fetches=[sol_loss],
#                        loss_callback=callback)

In [32]:
sess.run(sol_loss, feed_dict=tf_dict)

0.00046073573

In [33]:
with open(__NAME + '/weights.pkl', 'wb') as db_file:
    pkl.dump(obj=sess.run(u_weights), file=db_file)

with open(__NAME + '/biases.pkl', 'wb') as db_file:
    pkl.dump(obj=sess.run(u_biases), file=db_file)

In [34]:
x_sol = np.linspace(lb[0], ub[0], num=1000)

In [35]:
u_pred = sess.run(u0_pred, {
        lb_tf: lb,
        ub_tf: ub,
        x0_tf: x_sol[:, None]
    })

In [36]:
# v_pred = sess.run(v0_pred, {
#         lb_tf: lb,
#         ub_tf: ub,
#         x0_tf: x_sol[:, None]
#     })

In [37]:
# f_pred = sess.run(sol_f1_pred, {
#         lb_tf: lb,
#         ub_tf: ub,
#         x_f_tf: x_sol[:, None]
#     })

In [38]:
# g_pred = sess.run(sol_g1_pred, {
#         lb_tf: lb,
#         ub_tf: ub,
#         x_f_tf: x_sol[:, None]
#     })

In [42]:
fig = plt.figure(figsize=(4*1.3333,4), dpi=200)
ax = fig.gca()

ax.set_xlim(lb[0], ub[0])
# ax.set_xticks(np.arange(lb[0],ub[0],(ub[0] - lb[0])/N))
# ax.set_yticks(np.arange(lb[1],ub[1],(ub[1] - lb[1])/N))

ax.yaxis.grid(color='gainsboro', linestyle='dotted', linewidth=1.5)
ax.xaxis.grid(color='gainsboro', linestyle='dotted', linewidth=0.8)
ax.axhline(0,linestyle='dotted', color='grey')
ax.spines['right'].set_color('none')
ax.spines['top'].set_color('none')
# plt.subplots_adjust(bottom=0.17)
# plt.subplots_adjust(left=0.17)

# plt.title('Simple Harmonic Oscillator')
plt.xlabel('$x$')
plt.ylabel('$y$')

ax.plot(x_sol, u_pred, color ='red', label='PINN sol')
ax.plot(x_sol, 3 * np.exp(-x_sol), color ='blue', label='ideal sol', ls='-.')

plt.tight_layout()

plt.legend()

# fig.show()
fig.savefig('Figures\\SHO 2.png')

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [45]:
np.abs(1/u_pred).argmax()

531

In [48]:
x_sol[530]

3.33342163443962

In [47]:
u_pred[530]

array([0.00120939], dtype=float32)